# Text-to-SQL

In [22]:
import json

from rich import print

from guardrails.applications.text2sql import Text2Sql
from guardrails.llm_providers import openai_wrapper

## Set up SQL information

In [16]:
EXAMPLES = "/Users/shreyarajpal/guardrails/tests/integration_tests/test_assets/text2sql/examples.json"
SQL_SCHEMA = "/Users/shreyarajpal/guardrails/tests/integration_tests/test_assets/text2sql/schema.sql"
SQL_CONN = "sqlite:////Users/shreyarajpal/guardrails/tests/integration_tests/test_assets/text2sql/department_management.sqlite"

In [29]:
with open(EXAMPLES) as f:
    examples = json.load(f)

print(examples)

[
    {
        'question': 'How many heads of the departments are older than 56 ?',
        'query': 'SELECT count(*) FROM head WHERE age  >  56'
    },
    {
        'question': 'List the name, born state and age of the heads of departments ordered by age.',
        'query': 'SELECT name ,  born_state ,  age FROM head ORDER BY age'
    },
    {
        'question': 'List the creation year, name and budget of each department.',
        'query': 'SELECT creation ,  name ,  budget_in_billions FROM department'
    },
    {
        'question': 'What are the maximum and minimum budget of the departments?',
        'query': 'SELECT max(budget_in_billions) ,  min(budget_in_billions) FROM department'
    },
    {
        'question': 'What is the average number of employees of the departments whose rank is between 10 and 15?',
        'query': 'SELECT avg(num_employees) FROM department WHERE ranking BETWEEN 10 AND 15'
    },
    {
        'question': 'What are the names of the heads who are born outside the California state?',
        'query': "SELECT name FROM head WHERE born_state != 'California'"
    },
    {
        'question': "What are the distinct creation years of the departments managed by a secretary born in state 
'Alabama'?",
        'query': "SELECT DISTINCT T1.creation FROM department AS T1 JOIN management AS T2 ON T1.department_id  =  
T2.department_id JOIN head AS T3 ON T2.head_id  =  T3.head_id WHERE T3.born_state  =  'Alabama'"
    },
    {
        'question': 'What are the names of the states where at least 3 heads were born?',
        'query': 'SELECT born_state FROM head GROUP BY born_state HAVING count(*)  >=  3'
    },
    {
        'question': 'In which year were most departments established?',
        'query': 'SELECT creation FROM department GROUP BY creation ORDER BY count(*) DESC LIMIT 1'
    },
    {
        'question': "Show the name and number of employees for the departments managed by heads whose temporary 
acting value is 'Yes'?",
        'query': "SELECT T1.name ,  T1.num_employees FROM department AS T1 JOIN management AS T2 ON 
T1.department_id  =  T2.department_id WHERE T2.temporary_acting  =  'Yes'"
    },
    {
        'question': 'How many acting statuses are there?',
        'query': 'SELECT count(DISTINCT temporary_acting) FROM management'
    },
    {
        'question': 'How many departments are led by heads who are not mentioned?',
        'query': 'SELECT count(*) FROM department WHERE department_id NOT IN (SELECT department_id FROM 
management);'
    },
    {
        'question': 'What are the distinct ages of the heads who are acting?',
        'query': "SELECT DISTINCT T1.age FROM management AS T2 JOIN head AS T1 ON T1.head_id  =  T2.head_id WHERE 
T2.temporary_acting  =  'Yes'"
    },
    {
        'question': "List the states where both the secretary of 'Treasury' department and the secretary of 
'Homeland Security' were born.",
        'query': "SELECT T3.born_state FROM department AS T1 JOIN management AS T2 ON T1.department_id  =  
T2.department_id JOIN head AS T3 ON T2.head_id  =  T3.head_id WHERE T1.name  =  'Treasury' INTERSECT SELECT 
T3.born_state FROM department AS T1 JOIN management AS T2 ON T1.department_id  =  T2.department_id JOIN head AS T3 
ON T2.head_id  =  T3.head_id WHERE T1.name  =  'Homeland Security'"
    },
    {
        'question': 'Which department has more than 1 head at a time? List the id, name and the number of heads.',
        'query': 'SELECT T1.department_id ,  T1.name ,  count(*) FROM management AS T2 JOIN department AS T1 ON 
T1.department_id  =  T2.department_id GROUP BY T1.department_id HAVING count(*)  >  1'
    },
    {
        'question': "Which head's name has the substring 'Ha'? List the id and name.",
        'query': "SELECT head_id ,  name FROM head WHERE name LIKE '%Ha%'"
    }
]

## Use Text2SQL Application

- Sets up sandboxed DB based on your SQL schema / connection string
- Finds most relevant examples and inserts them into the prompt
- Checks that the query is valid for the schema

In [30]:
app = Text2Sql(
    "sqlite://",
    schema_file=SQL_SCHEMA,
    examples=examples
)

In [31]:
print(app("What is the name of the department with the highest number of employees?"))

SELECT Name FROM department ORDER BY Num_Employees DESC LIMIT 1

In [32]:
app.guard.state.most_recent_call.tree

Logs
└── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
    │ ╭──────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────╮ │
    │ │                                                                                                         │ │
    │ │ You are a data scientist whose job is to write SQL queries.                                             │ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ │ Given below is XML that describes the information to extract from this document and the tags to extract │ │
    │ │ it into.                                                                                                │ │
    │ │                                                                                                         │ │
    │ │ <output>                                                                                                │ │
    │ │     <sql name="generated_sql" description="Generate SQL for the given natural language instruction."    │ │
    │ │ format="bug-free-sql"/>                                                                                 │ │
    │ │ </output>                                                                                               │ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ │ ONLY return a valid JSON object (no other text is necessary), where the key of the field in JSON is the │ │
    │ │ `name` attribute of the corresponding XML, and the value is of the type specified by the corresponding  │ │
    │ │ XML's tag. The JSON MUST conform to the XML format, including any types and format requests e.g.        │ │
    │ │ requests for lists, objects and specific types. Be correct and concise.                                 │ │
    │ │                                                                                                         │ │
    │ │ Here are examples of simple (XML, JSON) pairs that show the expected behavior:                          │ │
    │ │ - `<string name='foo' format='two-words lower-case' />` => `{'foo': 'example one'}`                     │ │
    │ │ - `<list name='bar'><string format='upper-case' /></list>` => `{"bar": ['STRING ONE', 'STRING TWO',     │ │
    │ │ etc.]}`                                                                                                 │ │
    │ │ - `<object name='baz'><string name="foo" format="capitalize two-words" /><integer name="index"          │ │
    │ │ format="1-indexed" /></object>` => `{'baz': {'foo': 'Some String', 'index': 1}}`                        │ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ │ Here's schema about the database that you can use to generate the SQL query.                            │ │
    │ │ Try to avoid using joins if the data can be retrieved from the same table.                              │ │
    │ │                                                                                                         │ │
    │ │ Table: department                                                                                       │ │
    │ │     Column: Department_ID                                                                               │ │
    │ │         type: INTEGER                                                                                   │ │
    │ │     Column: Name                           

## Testing this on an incorrect output

In [33]:
llm_output = '{"generated_sql": "SELECT name FROM departments ORDER BY num_employees DESC LIMIT 1"}'

In [34]:
output = app.guard.parse(
    llm_output=llm_output,
    llm_api=openai_wrapper,
    engine="text-davinci-003",
    prompt_params={
        "nl_instruction": "What is the name of the department with the highest number of employees?",
        "db_info": str(app.sql_schema),
        "examples": "",
    },
    max_tokens=512,
)

In [35]:
print(output['generated_sql'])

SELECT Name FROM department ORDER BY Num_Employees DESC LIMIT 1

In [36]:
app.guard.state.most_recent_call.tree

Logs
├── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
│   │ ╭──────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────╮ │
│   │ │ No prompt                                                                                               │ │
│   │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│   │ ╭──────────────────────────────────────────── Raw LLM Output ─────────────────────────────────────────────╮ │
│   │ │ {"generated_sql": "SELECT name FROM departments ORDER BY num_employees DESC LIMIT 1"}                   │ │
│   │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│   │ ╭─────────────────────────────────────────── Validated Output ────────────────────────────────────────────╮ │
│   │ │ {                                                                                                       │ │
│   │ │     'generated_sql': ReAsk(                                                                             │ │
│   │ │         incorrect_value='SELECT name FROM departments ORDER BY num_employees DESC LIMIT 1',             │ │
│   │ │         error_message='(sqlite3.OperationalError) no such table: departments\n[SQL: SELECT name FROM    │ │
│   │ │ departments ORDER BY num_employees DESC LIMIT 1]\n(Background on this error at:                         │ │
│   │ │ https://sqlalche.me/e/20/e3q8)',                                                                        │ │
│   │ │         fix_value=None,                                                                                 │ │
│   │ │         path=['generated_sql']                                                                          │ │
│   │ │     )                                                                                                   │ │
│   │ │ }                                                                                                       │ │
│   │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│   ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
└── ╭────────────────────────────────────────────────── Step 1 ───────────────────────────────────────────────────╮
    │ ╭──────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────╮ │
    │ │                                                                                                         │ │
    │ │ You are a data scientist whose job is to write SQL queries.                                             │ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ │ Given below is XML that describes the information to extract from this document and the tags to extract │ │
    │ │ it into.                                                                                                │ │
    │ │                                                                                                         │ │
    │ │ <output>                                                                                                │ │
    │ │     <sql name="generated_sql" description="Generate SQL for the given natural language instruction."    │ │
    │ │ format="bug-free-sql"/>                                                                                 │ │
    │ │ </output>                                                                                               │ │
    │ │                                                                                                         │ │
    │ │                                            